# Setup

In [1]:
!pip3 install torchmetrics transformers accelerate datasets deepspeed wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.5 MB/s

In [2]:
!nvidia-smi

Mon Oct 30 08:53:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
import torch
from torchmetrics import Accuracy, Precision, Recall, F1Score, AUROC
from transformers import Trainer, TrainingArguments, EvalPrediction
import wandb
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset


In [4]:
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '9994' # modify if RuntimeError: Address already in use
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"

# Load Dataset

In [5]:
dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

# Models

In [19]:
# https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english
distilbertTokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
distilbertModel = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# https://huggingface.co/siebert/sentiment-roberta-large-english
robertaTokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
robertaModel = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")

# Function to generate train & eval set

In [15]:
def generate_datasets(tokenizer):
  # create tokenizer function
  def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
  # create datasets
  tokenized_datasets = dataset.map(tokenize_function, batched=True)
  train_dataset = tokenized_datasets["train"]
  eval_dataset = tokenized_datasets["test"]
  return train_dataset, eval_dataset

# Create Deep Speed config

In [9]:
%%bash
cat <<'EOT' > ds_config.json
{
  "zero_optimization": {
    "stage": 2,
    "offload_optimizer": {
      "device": "cpu",
      "pin_memory": true
    },
    "allgather_partitions": true,
    "allgather_bucket_size": 2e8,
    "reduce_scatter": true,
    "reduce_bucket_size": 2e8,
    "overlap_comm": true,
    "contiguous_gradients": true
  },

  "zero_allow_untested_optimizer": true,

  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": "auto",
      "betas": "auto",
      "eps": "auto",
      "weight_decay": "auto"
    }
  },

  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": "auto",
      "warmup_max_lr": "auto",
      "warmup_num_steps": "auto"
    }
  },

  "gradient_accumulation_steps": "auto",
  "gradient_clipping": "auto",
  "steps_per_print": 10,
  "train_batch_size": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "wall_clock_breakdown": false
}
EOT

# Trainer

In [11]:

os.environ["WANDB_ENTITY"] = "sc4001" # name of W&B team
os.environ["WANDB_PROJECT"] = "text-sentiment-analysis" # name of W&B project

wandb.login()

# default optimizer: AdamW
training_args = TrainingArguments(
    output_dir='./results', # output directory of results
    num_train_epochs=3, # number of train epochs
    report_to='wandb', # enable logging to W&B
    evaluation_strategy='steps', # check evaluation metrics at each epoch
    logging_steps = 10, # we will log every 10 steps
    eval_steps = 50, # we will perform evaluation every 50 steps
    save_steps = 50, # we will save the model every 50 steps
    load_best_model_at_end = True, # we will load the best model at the end of training
    metric_for_best_model = 'accuracy', # metric to see which model is better
    deepspeed='ds_config.json', # deep speed integration

    #### effective batch_size = per_device_train_batch_size x gradient_accumulation_steps ####
    #### We set effective batch_size to 32 ####
    per_device_train_batch_size=8, # batch size per device
    gradient_accumulation_steps=4, # gradient accumulation
    per_device_eval_batch_size=8, # eval batch size per device
)


def compute_metrics(pred: EvalPrediction):
    """
    Compute metrics using torchmetrics for a given set of predictions and labels.

    Args:
    pred (EvalPrediction): An object containing model predictions and labels.

    Returns:
    dict: A dictionary containing metric results.
    """
    # Extract labels and predictions
    labels = pred.label_ids
    preds = pred.predictions

    num_classes = preds.shape[1]

    # Convert to torch tensors
    labels = torch.tensor(labels)
    preds = torch.tensor(preds)

    # Initialize metrics
    accuracy = Accuracy(task="multiclass", num_classes=num_classes)
    precision = Precision(task="multiclass", num_classes=num_classes)
    recall = Recall(task="multiclass", num_classes=num_classes)
    f1 = F1Score(task="multiclass", num_classes=num_classes)
    auroc = AUROC(task="multiclass", num_classes=num_classes)

    # Calculate metrics (automatically does argmax)
    accuracy_score = accuracy(preds, labels)
    precision_score = precision(preds, labels)
    recall_score = recall(preds, labels)
    f1_score = f1(preds, labels)
    auroc_score = auroc(preds, labels)


    # Convert to CPU for serialization
    return {
        "accuracy": accuracy_score.cpu().item(),
        "precision": precision_score.cpu().item(),
        "recall": recall_score.cpu().item(),
        "f1": f1_score.cpu().item(),
        "auroc": auroc_score.cpu().item(),
    }

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, compute_metrics=compute_metrics, args=training_args, **kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        Override the default compute_loss.
        Use Cross Entropy Loss for multiclass classification (>= 2).
        """
        labels = inputs.pop("labels")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute cross entropy loss
        loss_func = torch.nn.CrossEntropyLoss()
        loss = loss_func(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[2023-10-30 09:01:05,813] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-10-30 09:01:06,480] [INFO] [comm.py:637:init_distributed] cdb=None
[2023-10-30 09:01:06,484] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


# Fine-tuning pre-trained roberta model

In [18]:
r_train_dataset, r_eval_dataset = generate_datasets(robertaTokenizer)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
r_trainer = CustomTrainer(
    model=robertaModel,
    train_dataset=r_train_dataset,
    eval_dataset=r_eval_dataset,
)

r_trainer.train()

Using /root/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
No modifications detected for re-loaded extension module cpu_adam, skipping build step...
Loading extension module cpu_adam...


Time to load cpu_adam op: 2.4242541790008545 seconds


# Fine-tuning pre-trained DistilBERTa model

In [ ]:
d_train_dataset, d_eval_dataset = generate_datasets(distilbertTokenizer)

In [ ]:
d_trainer = CustomTrainer(
    model=distilbertModel,
    train_dataset=d_train_dataset,
    eval_dataset=d_eval_dataset,
)

d_trainer.train()